In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
from threading import Thread

class StreamingThread(Thread):
    def __init__(self, ssc):
        Thread.__init__(self)
        self.ssc = ssc
    def run(self):
        ssc.start()
        ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
spark

In [5]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType

In [6]:
globals()['models_loaded'] = False
globals()['my_model'] = None

# Very simple predict function. Normally you'd use your loaded globals()['my_model'] here
def predict(df):
    l = ["vaccine", "stopasianhate", "covid", "china", "inflation", "biden"]
    for word in l:
        if word in df.tweet_text.lower():
            return word
    else:
        return 'covid'

predict_udf = udf(predict, StringType())

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()
    
    # Utilize our predict function
    df_withpreds = df.withColumn("pred", predict_udf(
        struct([df[x] for x in df.columns])
    ))
    df_withpreds.show()
    
    # Normally, you wouldn't use a UDF (User Defined Function) Python function to predict as we did here (you can)
    # but an MLlib model you've built and saved with Spark
    # In this case, you need to prevent loading your model in every call to "process" as follows:
    
    # Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # load in your models here
        globals()['my_model'] = '***' # Replace '***' with:    [...].load('my_logistic_regression')
        globals()['models_loaded'] = True
        
    # And then predict using the loaded model: 
    # df_result = globals()['my_model'].transform(df)
    # df_result.show()

In [7]:
ssc = StreamingContext(sc, 10)

In [8]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

In [9]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

========= 2021-04-06 12:12:00 =========
+--------------+-------------------+--------------------+
|         label|           tweet_id|          tweet_text|
+--------------+-------------------+--------------------+
|#stopasianhate|1379373968864407562|😩😩😩😩
GIVE A V...|
+--------------+-------------------+--------------------+

+--------------+-------------------+--------------------+-----+
|         label|           tweet_id|          tweet_text| pred|
+--------------+-------------------+--------------------+-----+
|#stopasianhate|1379373968864407562|😩😩😩😩
GIVE A V...|covid|
+--------------+-------------------+--------------------+-----+

========= 2021-04-06 12:12:10 =========
+------+-------------------+--------------------+
| label|           tweet_id|          tweet_text|
+------+-------------------+--------------------+
|#covid|1379374275728191490|Stand-Up Comedian...|
+------+-------------------+--------------------+

+------+-------------------+--------------------+-----+
| label| 

In [10]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
+------+-------------------+--------------------+
| label|           tweet_id|          tweet_text|
+------+-------------------+--------------------+
|#covid|1379374889824583681|For the latest in...|
+------+-------------------+--------------------+

+------+-------------------+--------------------+-----+
| label|           tweet_id|          tweet_text| pred|
+------+-------------------+--------------------+-----+
|#covid|1379374889824583681|For the latest in...|covid|
+------+-------------------+--------------------+-----+

